In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

# ตั้งค่าให้แสดงผล Pandas สวยงาม
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

# รายชื่อหุ้น SET50 (Hardcode ไว้ใน cell นี้เพื่อให้รันได้เลยโดยไม่ต้องพึ่งไฟล์ config.py)
SET50_TICKERS = [
    "ADVANC.BK", "AOT.BK", "AWC.BK", "BANPU.BK", "BBL.BK", "BDMS.BK", "BEM.BK", "BGRIM.BK", "BH.BK", "BJC.BK",
    "BPP.BK", "CPALL.BK", "CPF.BK", "CPN.BK", "CRC.BK", "DELTA.BK", "EGCO.BK", "GULF.BK", "HMPRO.BK",
    "IRPC.BK", "KBANK.BK", "KTB.BK", "KTC.BK", "LH.BK", "MINT.BK", "MTC.BK", "OR.BK", "OSP.BK",
    "PTT.BK", "PTTEP.BK", "PTTGC.BK", "RATCH.BK", "SAWAD.BK", "SCB.BK", "SCC.BK", "SCGP.BK", "TISCO.BK", "TLI.BK",
    "TOP.BK", "TTB.BK", "TU.BK", "VGI.BK", "WHA.BK", "GLOBAL.BK", "BAM.BK", "CPAXT.BK", "GPSC.BK", "BLA.BK"
]

print(f"✅ Loaded {len(SET50_TICKERS)} stocks from SET50.")

✅ Loaded 48 stocks from SET50.


In [11]:
# แปลง Day of Year (1-366) เป็น 'Month-Day' (เช่น 'Apr-25')
def dayofyear_to_str(doy, year=2000):
    if pd.isna(doy): return None
    base = pd.Timestamp(f'{year}-01-01')
    return (base + pd.to_timedelta(int(round(doy))-1, unit='D')).strftime('%b-%d')

# ฟังก์ชันติด Tag ปันผล (แบ่งตามครึ่งปีแรก/หลัง)
def tag_dividends_per_year(df_div):
    """
    ติด Tag โดยแบ่งตามช่วงเดือน (Seasonality)
    Tag 1: ช่วงครึ่งปีแรก (Jan - Jun)
    Tag 2: ช่วงครึ่งปีหลัง (Jul - Dec)
    """
    df = df_div.copy()
    df = df[df['Dividends'] != 0].copy()
    if df.empty: return df
    
    idx = pd.to_datetime(df.index)
    if idx.tz is not None: idx = idx.tz_convert(None)
    df.index = idx
    
    df['year'] = df.index.year
    df['month'] = df.index.month
    df = df.sort_index()
    
    # [NEW LOGIC] แบ่งตามเดือนแทนลำดับ
    df['tag'] = np.where(df['month'] <= 6, 1, 2)
    
    return df

# ฟังก์ชันคำนวณวันนับถอยหลัง
def calculate_days_to_dividend(avg_month_day_str):
    """คำนวณวันนับถอยหลัง (Countdown)"""
    if not avg_month_day_str: return None
    
    current_date = datetime.now()
    current_year = current_date.year
    
    try:
        # ลองเทียบกับปีนี้
        target_date = datetime.strptime(f"{current_year}-{avg_month_day_str}", "%Y-%b-%d")
        diff = (target_date - current_date).days
        
        # ถ้าเลยวันไปแล้ว (+1 ปี) -> รอปีหน้า
        if diff < 0:
            target_next = datetime.strptime(f"{current_year + 1}-{avg_month_day_str}", "%Y-%b-%d")
            diff = (target_next - current_date).days
            
        return diff
    except ValueError:
        return None

In [12]:
def analyze_stock_seasonality(symbol: str):
    """
    วิเคราะห์หุ้นรายตัว: หา Min, Max, Avg Date และ Countdown
    """
    try:
        clean_symbol = symbol.upper().replace('.BK', '')
        ticker = f"{clean_symbol}.BK"
        
        stock = yf.Ticker(ticker)
        # ดึง 10 ปีเพื่อให้เห็น Min/Max ที่มีความหมาย
        hist = stock.history(period="10y") 
        
        if hist.empty:
            return None
            
        div_df = hist[['Dividends']]
        tagged_df = tag_dividends_per_year(div_df)
        
        if tagged_df.empty or 'tag' not in tagged_df.columns:
            return None

        result_data = {'Symbol': clean_symbol}
        
        # วนลูปหา Tag 1 (ต้นปี) และ Tag 2 (กลางปี/ท้ายปี)
        for t in [1, 2]:
            subset = tagged_df[tagged_df['tag'] == t].copy()
            
            if not subset.empty:
                # [FIXED] แปลง Index เป็น Series ก่อนคำนวณ .mean()
                doy_values = subset.index.dayofyear
                doy_series = pd.Series(doy_values)
                
                # ตอนนี้คำนวณได้แล้ว
                avg_str = dayofyear_to_str(doy_series.mean())
                min_str = dayofyear_to_str(doy_series.min())
                max_str = dayofyear_to_str(doy_series.max())
                
                # คำนวณ Countdown
                countdown = calculate_days_to_dividend(avg_str)
                
                result_data[f'Tag{t}'] = {
                    "Stats": {
                        "Min_Date": min_str,
                        "Max_Date": max_str,
                        "Avg_Date": avg_str,
                        "Data_Points": len(subset)
                    },
                    "Countdown": {
                        "Avg_Date": avg_str,
                        "Days_Remaining": countdown
                    }
                }
            else:
                result_data[f'Tag{t}'] = None
                
        return result_data

    except Exception as e:
        print(f"Error seasonality {symbol}: {e}")
        return None

In [13]:
# ทดสอบกับหุ้น PTT
test_symbol = "PTT.BK"
print(f"🔍 Analyzing Seasonality for {test_symbol}...")

result = analyze_stock_seasonality(test_symbol)

if result:
    print(f"\n✅ Result for {result['Symbol']}:")
    
    # แสดง Tag 1 (ปันผลครึ่งปีแรก)
    if result['Tag1']:
        print("\n[Tag 1: Dividend Season Jan-Jun]")
        print(f"  📅 Avg Date: {result['Tag1']['Stats']['Avg_Date']}")
        print(f"  📉 Min Date: {result['Tag1']['Stats']['Min_Date']}")
        print(f"  📈 Max Date: {result['Tag1']['Stats']['Max_Date']}")
        print(f"  ⏳ Countdown: {result['Tag1']['Countdown']['Days_Remaining']} days left")
    else:
        print("\n[Tag 1] No data")

    # แสดง Tag 2 (ปันผลครึ่งปีหลัง)
    if result['Tag2']:
        print("\n[Tag 2: Dividend Season Jul-Dec]")
        print(f"  📅 Avg Date: {result['Tag2']['Stats']['Avg_Date']}")
        print(f"  📉 Min Date: {result['Tag2']['Stats']['Min_Date']}")
        print(f"  📈 Max Date: {result['Tag2']['Stats']['Max_Date']}")
        print(f"  ⏳ Countdown: {result['Tag2']['Countdown']['Days_Remaining']} days left")
    else:
        print("\n[Tag 2] No data")

🔍 Analyzing Seasonality for PTT.BK...

✅ Result for PTT:

[Tag 1: Dividend Season Jan-Jun]
  📅 Avg Date: Mar-05
  📉 Min Date: Feb-28
  📈 Max Date: Mar-27
  ⏳ Countdown: 84 days left

[Tag 2: Dividend Season Jul-Dec]
  📅 Avg Date: Sep-21
  📉 Min Date: Aug-27
  📈 Max Date: Oct-09
  ⏳ Countdown: 284 days left


In [14]:
results_list = []

print(f"🚀 Batch Processing for {len(SET50_TICKERS)} stocks...")

for symbol in SET50_TICKERS:
    data = analyze_stock_seasonality(symbol)
    if data:
        row = {'Symbol': data['Symbol']}
        
        # ดึงข้อมูล Tag 1
        if data.get('Tag1'):
            row['Tag1_Avg'] = data['Tag1']['Stats']['Avg_Date']
            row['Tag1_Countdown'] = data['Tag1']['Countdown']['Days_Remaining']
        else:
            row['Tag1_Avg'] = None
            row['Tag1_Countdown'] = 9999 # ค่าสมมติเพื่อให้ sort ไปอยู่ท้ายๆ

        # ดึงข้อมูล Tag 2
        if data.get('Tag2'):
            row['Tag2_Avg'] = data['Tag2']['Stats']['Avg_Date']
            row['Tag2_Countdown'] = data['Tag2']['Countdown']['Days_Remaining']
        else:
            row['Tag2_Avg'] = None
            row['Tag2_Countdown'] = 9999
            
        results_list.append(row)

# สร้าง DataFrame
df_results = pd.DataFrame(results_list)

# เรียงลำดับตามหุ้นที่ใกล้ปันผล Tag 1 มากที่สุด
df_sorted = df_results.sort_values(by='Tag1_Countdown')

print("\n📋 Top 10 Stocks closest to Tag 1 Dividend (Jan-Jun):")
display(df_sorted[['Symbol', 'Tag1_Avg', 'Tag1_Countdown', 'Tag2_Avg', 'Tag2_Countdown']].head(10))

🚀 Batch Processing for 48 stocks...

📋 Top 10 Stocks closest to Tag 1 Dividend (Jan-Jun):


,Symbol,Tag1_Avg,Tag1_Countdown,Tag2_Avg,Tag2_Countdown
28,PTTEP,Feb-13,64.0,Aug-09,241
40,VGI,Feb-20,71.0,Jul-18,219
18,IRPC,Feb-23,74.0,Sep-13,276
45,GPSC,Feb-24,75.0,Sep-01,264
37,TOP,Feb-26,77.0,Sep-15,278
29,PTTGC,Feb-27,78.0,Aug-30,262
25,OR,Feb-28,79.0,Sep-03,266
0,ADVANC,Mar-02,81.0,Aug-15,247
42,GLOBAL,Mar-02,81.0,None,9999
39,TU,Mar-03,82.0,Aug-19,251
